In [ ]:
"""cifar_100tl.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1f5Vu-ZkMtnS_KGN4b7nK1A9j1Cpjvgit

## Using Resnet50 for Transfer Learning and classify the Cifar - 100 data

### The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images
"""

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
"""## Importing Libraries"""

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from datetime import datetime
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, UpSampling2D, Dropout,BatchNormalization,GlobalAveragePooling2D
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

In [ ]:
"""## Importing the Cifar 100 Dataset"""

In [ ]:
cifar100 = tf.keras.datasets.cifar100
(X_train, Y_train), (X_test,Y_test) = cifar100.load_data()

In [ ]:
"""##  Plotting some images from the dataset"""

In [ ]:
plt.figure(figsize=(12,12))
for i in range(100):
  plt.subplot(10,10,1+i)
  plt.axis('off')
  plt.imshow(X_train[i],cmap='gray')

In [ ]:
"""## Splitting the train data again - we use the val set as test set and previous test set for final predictions"""

In [ ]:
X_train,X_val,y_train,y_val = train_test_split(X_train, Y_train, test_size = 0.2)

In [ ]:
"""##  Onehot encoding of the outputs"""

In [ ]:
y_train = to_categorical(y_train, num_classes = 100)
y_val = to_categorical(y_val, num_classes = 100)
y_test = to_categorical(Y_test, num_classes = 100)

In [ ]:
X_train = X_train * 1.0/255
x_val = X_val * 1.0/255
X_test = X_test * 1.0/255

In [ ]:
print(X_train.shape, x_val.shape, X_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)

In [ ]:
"""##  Image Data Augmentation"""

In [ ]:
train_datagen = ImageDataGenerator(
        rotation_range = 10,
        zoom_range = 0.1,
        width_shift_range = 0.1,
        height_shift_range = 0.1,
        shear_range = 0.1,
        horizontal_flip = True,
        vertical_flip = False
        )
train_datagen.fit(X_train)

In [ ]:
"""##  Reduce Learning Rate if accuracy is not improving for 3 epochs"""

In [ ]:
from keras.callbacks import ReduceLROnPlateau
learning_rate_reduction = ReduceLROnPlateau(
    monitor='val_accuracy',
    patience=3,
    verbose=1,
    factor=0.6,
    min_lr=1e-6)

In [ ]:
"""##  Importing the Resnet Model"""

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
resnet_model = ResNet50(
    include_top = False,
    weights = 'imagenet',
    input_shape = (224,224,3)
)

In [ ]:
for layer in resnet_model.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
resnet_model.summary()

In [ ]:
"""##  Converting the output layer as per our dataset"""

In [ ]:
model=tf.keras.models.Sequential()
model.add(UpSampling2D(size=(7, 7),interpolation='bilinear'))
model.add(resnet_model)
model.add(GlobalAveragePooling2D())
model.add(Dropout(.25))
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(100, activation='softmax'))

In [ ]:
"""Keras provides an upsampling layer – called UpSampling2D – which allows to perform upsampling operation within neural networks
* ### 32 * 7 = 224

### Stochastic gradient descent optimizer with momentum.
"""

In [ ]:
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3, momentum=0.9)

In [ ]:
"""Compile the model"""

In [ ]:
model.compile(
    optimizer = optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
"""We now Train the model on images. we are also checking to see if validation accuracy doesnt improve we will reduce learning rate though the callback"""

In [ ]:
history = model.fit(
    train_datagen.flow(X_train, y_train, batch_size = 32),
    validation_data = (x_val, y_val),
    epochs = 25,
    verbose = 1,
    callbacks = [learning_rate_reduction]
)
model.save("/content/drive/MyDrive/machine learning.h5")

In [ ]:
model.save("cifar100.h5")

In [ ]:
model.summary()

In [ ]:
"""## Plot accuracy and Loss"""

In [ ]:
"""## Predictions,Accuracy and Confusion Matrix"""

In [ ]:
y_pred = np.argmax(model.predict(X_test), axis=-1)
y_true = Y_test.ravel()
print(y_pred.shape,y_true.shape)

In [ ]:
print("Testing Accuracy: ", accuracy_score(y_true,y_pred))

In [ ]:
cm = confusion_matrix(y_true,y_pred)
cm

In [ ]:
# Define the class names for CIFAR-100
cifar100_class_names = [
    'apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle',
    'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle',
    'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup',
    'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house',
    'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man',
    'maple_tree', 'motorcycle', 'mountain', 'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid',
    'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine',
    'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea', 'seal', 'shark', 'shrew',
    'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper',
    'table', 'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout', 'tulip', 'turtle',
    'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman', 'worm'
]

In [ ]:
# Print class label names along with their corresponding class indices
for index, label_name in enumerate(cifar100_class_names):
   classes=  print(f"{index}: {label_name}")
print(classes)

In [ ]:
import numpy as np
import cv2
from keras.models import load_model
import matplotlib.pyplot as plt

In [ ]:
# Load your pre-trained model
model = load_model('/content/drive/MyDrive/machine learning.h5')

In [ ]:
# Load and preprocess the test image
test_img = cv2.imread('/content/photo-1558981285-6f0c94958bb6.jfif')
img = cv2.resize(test_img, (32, 32), interpolation=cv2.INTER_CUBIC)
img = np.reshape(img, (1, 32, 32, 3))
img = img / 255.0

In [ ]:
# Make predictions
pred = model.predict(img)
class_num = np.argmax(pred)

In [ ]:
# Define your class names (specific to your dataset)
cifar100_class_names = [
    'apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 'bed', 'bee', 'beetle', 'bicycle', 'bottle',
    'bowl', 'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 'castle', 'caterpillar', 'cattle',
    'chair', 'chimpanzee', 'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 'cup',
    'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 'fox', 'girl', 'hamster', 'house',
    'kangaroo', 'keyboard', 'lamp', 'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man',
    'maple_tree', 'motorcycle', 'mountain', 'mouse', 'mushroom', 'oak_tree', 'orange', 'orchid',
    'otter', 'palm_tree', 'pear', 'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine',
    'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea', 'seal', 'shark', 'shrew',
    'skunk', 'skyscraper', 'snail', 'snake', 'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper',
    'table', 'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout', 'tulip', 'turtle',
    'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman', 'worm'
] # Replace with your actual class names

In [ ]:
# Get the predicted class name
predicted_class_name = cifar100_class_names[class_num]

In [ ]:
# Display the image
plt.imshow(cv2.cvtColor(test_img, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for Matplotlib
plt.title(f"Predicted Class: {predicted_class_name}")
plt.axis('off')  # Hide axis
plt.show()

In [ ]:
# Print the predicted class name
print("The predicted class = ", predicted_class_name)